Here we will show an example of how to run deepblast on simulation data.

In [1]:
import os
from deepblast.sim import hmm_alignments
import argparse

We will first simulate multiple sequences from a single PFam family.
The resulting generated alignments will be used to train the model.

In [2]:
hmm = '../data/zf-C2H2.hmm'
n_alignments = 100
align_df = hmm_alignments(n=40, seed=0, n_alignments=n_alignments, hmmfile=hmm)

cols = [
    'chain1_name', 'chain2_name', 'tmscore1', 'tmscore2', 'rmsd',
    'chain1', 'chain2', 'alignment'
]
align_df.columns = cols

The simulated alignments will be split into training / testing and validation.

In [3]:
parts = n_alignments // 10
train_df = align_df.iloc[:parts * 8]
test_df = align_df.iloc[parts * 8:parts * 9]
valid_df = align_df.iloc[parts * 9:]

# save the files to disk.
train_df.to_csv('data/train.txt', sep='\t', index=None, header=None)
test_df.to_csv('data/test.txt', sep='\t', index=None, header=None)
valid_df.to_csv('data/valid.txt', sep='\t', index=None, header=None)

We will prepare the environment to make sure that the appropriate output directories exist to store the results.

In [4]:
from deepblast.trainer import LightningAligner
from pytorch_lightning import Trainer
from pytorch_lightning.profiler import AdvancedProfiler

output_dir = 'simulation_results'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [5]:
os.getcwd()

'/home/juermieboop/Documents/research/garfunkel/ipynb'

We will now create the arguments.  Below is the way to create this in a python environment.
This can also be recreated on a standard command line interface.

In [6]:
args = [
    '--train-pairs', f'{os.getcwd()}/data/train.txt',
    '--test-pairs', f'{os.getcwd()}/data/test.txt',
    '--valid-pairs', f'{os.getcwd()}/data/valid.txt',
    '--output-directory', output_dir,
    '--epochs', '10',
    '--batch-size', '50',    # this can only handle 1 batch atm. Need to change later.
    '--num-workers', '1',
    '--learning-rate', '1e-4',
    '--clip-ends', 'False',
    '--visualization-fraction', '0.5',
    '--gpus', '1'
]

We will then initialize the alignment model with the parameters we specified earlier.

In [7]:
parser = argparse.ArgumentParser(add_help=False)
parser = LightningAligner.add_model_specific_args(parser)
parser.add_argument('--num-workers', type=int)
parser.add_argument('--gpus', type=int)
args = parser.parse_args(args)
model = LightningAligner(args)

We can now train the model.

In [8]:
profiler = AdvancedProfiler()
trainer = Trainer(
    max_epochs=args.epochs,
    gpus=args.gpus,
    check_val_every_n_epoch=1,
    #profiler=profiler,
    fast_dev_run=False,
    # auto_scale_batch_size='power'
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                   | Params
---------------------------------------------------
0 | aligner | NeedlemanWunschAligner | 23 M  
/home/juermieboop/miniconda3/envs/pytorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/juermieboop/miniconda3/envs/pytorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/juermieboop/miniconda3/envs/pytorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



Profiler Report

Profile stats for: on_train_start
         48591 function calls (42753 primitive calls) in 0.024 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.024    0.024 callback_hook.py:64(on_train_start)
        1    0.000    0.000    0.024    0.024 progress.py:313(on_train_start)
        1    0.000    0.000    0.024    0.024 progress.py:264(init_train_tqdm)
        1    0.000    0.000    0.023    0.023 notebook.py:185(__init__)
        1    0.000    0.000    0.022    0.022 notebook.py:81(status_printer)
      8/3    0.000    0.000    0.013    0.004 widget.py:409(__init__)
      8/3    0.000    0.000    0.013    0.004 widget.py:425(open)
    19/14    0.000    0.000    0.012    0.001 widget.py:492(get_state)
3258/1931    0.001    0.000    0.011    0.000 traitlets.py:545(__get__)
 1286/901    0.000    0.000    0.011    0.000 traitlets.py:526(get)
8848/5172    0.002    0.000    0.011 

1

The model diagnostics can be directly visualized in Tensorboard. Here, we show the losses, the accuracy and the alignment results.

In [9]:
!ls lightning_logs

version_0   version_12	version_16  version_2	version_5  version_9
version_1   version_13	version_17  version_20	version_6
version_10  version_14	version_18  version_3	version_7
version_11  version_15	version_19  version_4	version_8


In [10]:
%load_ext tensorboard

In [11]:
%tensorboard --logdir lightning_logs

Reusing TensorBoard on port 6006 (pid 6845), started 0:09:05 ago. (Use '!kill 6845' to kill it.)

And we did this with just a few million parameters

In [12]:
model.aligner

NeedlemanWunschAligner(
  (lm): BiLM(
    (embed): Embedding(22, 21, padding_idx=21)
    (dropout): Dropout(p=0, inplace=False)
    (rnn): ModuleList(
      (0): LSTM(21, 1024, batch_first=True)
      (1): LSTM(1024, 1024, batch_first=True)
    )
    (linear): Linear(in_features=1024, out_features=21, bias=True)
  )
  (embedding): StackedRNN(
    (embed): Embedding(21, 512, padding_idx=20)
    (dropout): Dropout(p=0, inplace=False)
    (rnn): LSTM(512, 512, num_layers=2, batch_first=True, bidirectional=True)
    (proj): Linear(in_features=1024, out_features=512, bias=True)
  )
  (gap_score): Linear(in_features=1024, out_features=1, bias=True)
  (nw): NeedlemanWunschDecoder()
)